In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/caba_para_mapa.csv',error_bad_lines=False)

#tenia un nan nose porque
properati.dropna(inplace=True)

<IPython.core.display.Javascript object>

# RF Clasificador

In [11]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_divisa = preprocessing.LabelEncoder()
divisa=properati['currency']
le_divisa.fit(divisa)
properati['currency'] = le_divisa.transform(divisa)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

### Pruebo con np.histogram

In [12]:
cant_buckets = 10

#buckets
cantidad,rango = np.histogram(properati['price_per_m2'], bins=cant_buckets)
print rango
properati['categories_by_price']=pd.cut(properati['price_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_per_m2'],rango)
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

properati.groupby('categories_by_price').count()

[  1367.   11220.2  21073.4  30926.6  40779.8  50633.   60486.2  70339.4
  80192.6  90045.8  99899. ]


,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,...,expenses,properati_url,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range
categories_by_price,,,,,,,,,,,,,,,,,,,,,
0.0,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
1.0,46,46,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
2.0,406,406,406,406,406,406,406,406,406,406,...,406,406,406,406,406,406,406,406,406,406
3.0,1034,1034,1034,1034,1034,1034,1034,1034,1034,1034,...,1034,1034,1034,1034,1034,1034,1034,1034,1034,1034
4.0,595,595,595,595,595,595,595,595,595,595,...,595,595,595,595,595,595,595,595,595,595
5.0,240,240,240,240,240,240,240,240,240,240,...,240,240,240,240,240,240,240,240,240,240
6.0,78,78,78,78,78,78,78,78,78,78,...,78,78,78,78,78,78,78,78,78,78
7.0,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
8.0,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13


In [13]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],properati['place_name'], \
        properati['property_type'])
y = properati['categories_by_price']

### Pruebo con Quantile . NO MEJORO saltear

In [14]:
from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer(copy=False)

In [15]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['price_usd_per_m2']
print X[0]
X=scaler.fit_transform(X,y)
print X[0]

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

(7.0, 8.5)
[ 0.86886887  0.86636637]
True
True


### Pruebo con RobustScaler. NO MEJORO saltear

In [16]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler(copy=False)

In [17]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['price_usd_per_m2']
print X[0]
X=scaler.fit_transform(X,y)
print X[0]

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

(7.0, 8.5)
[ 1.1875      0.97560976]
True
True


### Busco hiper-parametros con random search

In [18]:
%%notify

from sklearn.model_selection import train_test_split
rf = RandomForestClassifier(n_jobs=-1)

warm_start=[True,False]
oob_score=[True,False]
max_features =["auto","sqrt","log2"]
n_estimators = np.arange(10,100,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"warm_start": warm_start,"max_features":max_features,"n_estimators":n_estimators,"oob_score":oob_score}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=10,n_iter=50) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:451: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:456: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


GridSearchCV duro 771.68 segundos para 50 candidatos a hyper-parametros.

Puesto: 1
Promedio training score: 0.599 (std: 0.034)
Promedio validation score: 0.287 (std: 0.132)
Promedio fit time: 0.982s
Hiper-parametros: {'warm_start': True, 'n_estimators': 90, 'oob_score': False, 'max_features': 'sqrt'}

Puesto: 2
Promedio training score: 0.599 (std: 0.034)
Promedio validation score: 0.285 (std: 0.134)
Promedio fit time: 0.399s
Hiper-parametros: {'warm_start': False, 'n_estimators': 30, 'oob_score': False, 'max_features': 'log2'}

Puesto: 2
Promedio training score: 0.599 (std: 0.035)
Promedio validation score: 0.285 (std: 0.131)
Promedio fit time: 1.471s
Hiper-parametros: {'warm_start': True, 'n_estimators': 80, 'oob_score': True, 'max_features': 'log2'}

Puesto: 4
Promedio training score: 0.599 (std: 0.036)
Promedio validation score: 0.285 (std: 0.130)
Promedio fit time: 0.614s
Hiper-parametros: {'warm_start': False, 'n_estimators': 30, 'oob_score': True, 'max_features': 'auto'}

Puesto

<IPython.core.display.Javascript object>

In [19]:
grid_search.score(X_test,y_test) # pruebo el estimador refiteado con el set de test

NameError: name 'grid_search' is not defined